Choosing a performance estimation method often depends on the data one is modelling. For example, when one can assume independence and an identical distribution(i.i.d.) among observations, **cross-validation** is typically the most appropriate method[1].

In the general machine learning modeling process, we usually use the k-fold cross-validation method of Fig.(a). However, time series data has temporal autocorrelation, so the test data cannot appear before the train data, otherwise it will cause data leakage！

Therefore, we have 3 built-in cross-validation strategies for time series forecasting tasks in HyperTS.

- First, as fig.(b) *preq-bls*: The data is split into n blocks. In the initial iteration, only the first two blocks are used, the first for training and the second for test. In the next iteration, the second block is merged with the first and the third block is used for test. This procedure continues until all blocks are tested.

- Second, as fig.(c) *preq-slid-bls*: Instead of merging the blocks after each iteration (growing window), one can forget the older blocks in a sliding window fashion. This idea is typically adopted when past data becomes deprecated, which is common in non-stationary environments.

- Third, as Fig.(d) *preq-bls-gap*: This illustrates a prequential approach applied in blocks, where a gap block is introduced. The rationale behind this idea is to increase the independence between training and test sets.

For time series classification and regression, because they are splited at the sample level, information leakage is not involved. We adopt the cross-validation method in Fig.(a).

![cv](./images/cv.png)

References

[1] Cerqueira V, Torgo L, Mozetič I. Evaluating time series forecasting models: An empirical study on performance estimation methods[J]. Machine Learning, 2020, 109(11): 1997-2028.

**Example of use:**

#### 1. Prepare Dataset

In [1]:
from hyperts.datasets import load_network_traffic
from sklearn.model_selection import train_test_split

In [2]:
df = load_network_traffic(univariate=True)
train_data, test_data = train_test_split(df, test_size=168, shuffle=False)

#### 2. Create Experiment and Run

In [3]:
from hyperts import make_experiment

Set the parameter ``cv=True``, ``num_folds`` controls the number of folds.

In [4]:
experiment = make_experiment(train_data=train_data.copy(),
                             task='forecast',
                             mode='dl',
                             timestamp='TimeStamp',
                             covariates=['HourSin', 'WeekCos', 'CBWD'],
                             forecast_train_data_periods=24*12,
                             max_trials=5,
                             cv=True,
                             num_folds=3)
model = experiment.run()

In [5]:
model.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('data_preprocessing',
                 TSFDataPreprocessStep(covariate_cleaner=CovariateTransformer(covariables=['HourSin',
                                                                                           'WeekCos',
                                                                                           'CBWD'],
                                                                              data_cleaner_args={'correct_object_dtype': False,
                                                                                                 'int_convert_to': 'str'}),
                                       covariate_cleaner__covariables=['HourSin',
                                                                       'WeekCos',
                                                                       'CBWD'],
                                       covariate_cleaner__data_cleaner_args={'correct_object_dtype': False,
                

#### 3. Infer and Evaluation

In [6]:
X_test, y_test = model.split_X_y(test_data.copy())
forecast = model.predict(X_test)
results = model.evaluate(y_true=y_test, y_pred=forecast)
results

,Metirc,Score
0,mae,0.5523
1,mse,0.4975
2,rmse,0.7053
3,mape,0.3192
4,smape,0.2533
